In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

# DOWNLOAD PRETRAINED MODEL AND TOKENIZER
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_base = AutoModel.from_pretrained("roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
roberta_base.config.hidden_size

768

In [3]:
# Remove the pooler layer
roberta_base.pooler = None

In [4]:
import numpy as np
import pandas as pd
import ast

train_path = "/content/emopillar_train_filtered.csv"       # enter path to train dataset
val_path = "/content/emopillar_validation_filtered.csv"         # enter path to validation dataset
test_path = "/content/emopillar_test_filtered.csv"        # enter path to test dataset


df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

In [ ]:
class EmoPillars_Dataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer):
        self.tokenizer = tokenizer
        self.data = data
        self.max_len = 64
        self.target_cols = [str(i) for i in range(28)]
        self.soft_target_cols = [str(f"{i}_exp") for i in range(28)]

    def __len__(self):
        return(len(self.data))

    def __getitem__(self, idx):   #__get_item__  --> __getitem__
        item = self.data.iloc[idx]
        text = str(item.utterance)
        encoding = self.tokenizer.encode_plus(text,
                                            add_special_tokens=True,
                                            truncation=True,
                                            return_tensors='pt',
                                            max_length=self.max_len,
                                            padding='max_length',
                                            return_attention_mask=True)

        target = torch.tensor(item[self.target_cols].values.astype('float32'))
        soft_target = torch.tensor(item[self.soft_target_cols].values.astype('float32'))

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "atten_masks": encoding["attention_mask"].squeeze(0),
            "hard_target": target,
            "soft_target": soft_target
        }

In [6]:
data = EmoPillars_Dataset(df_train, roberta_tokenizer)

In [7]:
len(data)

82750

In [8]:
data.__getitem__(1)   #__get_item__  --> __getitem__

{'input_ids': tensor([    0,   100,   240,     7,   465,  7330,  2413, 10913,     8,   146,
           123,   582,    13,    39, 45492,   219,     4,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1]),
 'atten_masks': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'label_names': "['desire', 'anger']",
 'expressiveness': '[1.0, 0.9]',
 'hard_target': tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'soft_target': ten

In [9]:
# Data Loaders
train_dataloader = DataLoader(EmoPillars_Dataset(df_train, roberta_tokenizer), batch_size=64, num_workers=4)
val_dataloader = DataLoader(EmoPillars_Dataset(df_val, roberta_tokenizer), batch_size=64, num_workers=4)
test_dataloader = DataLoader(EmoPillars_Dataset(df_test, roberta_tokenizer), batch_size=64, num_workers=4)

In [ ]:
import json
import torch

with open("label_embeddings.json", "r") as f:
    label_embeddings = json.load(f)

emo_emb=[]
for k, v in label_embeddings.items():
    emb = torch.tensor(v, dtype=torch.float32)
    emo_emb.append(emb)

emotion_bank = torch.cat(emo_emb, dim=0)

## Model Architecture Design

In [ ]:
import os
import math
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import AdamW
from torch.amp.autocast_mode import autocast
from torch.amp.grad_scaler import GradScaler
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score

In [ ]:
class Encoder(nn.Module):
    """
    Text Encoder:
    - Takes in tokenized text (from tokenizer)
    - Generates the text embedding vector
    """
    def __init__(self, base_encoder):
        super().__init__()
        self.encoder = base_encoder

    def forward(self, inputs):
        """
        inputs: tokenizer output dict (input_ids, attention_mask)
        """
        outputs = self.encoder(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]                                                                   # [B, T, H]

        atten_mask = inputs['attention_mask']                                                                           # [B, T]
        # Mean pooling text_embeddings
        atten_mask = atten_mask.unsqueeze(-1).float()
        pooled_text_emb = (last_hidden_state * atten_mask).sum(dim=1) / atten_mask.sum(dim=1).clamp(min=1e-9)           # [B, H]

        return {
            "text_embed": pooled_text_emb,
            "last_hidden_state": last_hidden_state,
            "atten_mask": atten_mask
        }

In [ ]:
class CrossAttentionModule(nn.Module):
    def __init__(self, hidden_size = 768):
        super().__init__()
        self.hidden_size = hidden_size

        # Multi-head cross attention
        self.cross_attn = nn.MultiheadAttention(
            embed_dim=self.hidden_size,
            num_heads=6,
            batch_first=True
        )
        self.rms_norm = nn.RMSNorm(hidden_size)

    def forward(self, encoder_out, emotion_bank):
        # cross-attention (query=text, key/value=emotion)
        attn_out, _ = self.cross_attn(
            query = encoder_out["last_hidden_state"],                                         # [B, T, 768]
            key = emotion_bank.unsqueeze(0).shape,                                            # [1, 28, 768]
            value = emotion_bank.unsqueeze(0).shape,                                          # [1, 28, 768]
        )

        # Fuse and pool
        fused_hidden_state = encoder_out["last_hidden_state"] + attn_out
        atten_mask = encoder_out["atten_mask"]
        fused_emo_text_emb = (fused_hidden_state * atten_mask).sum(dim=1) / atten_mask.sum(dim=1).clamp(min=1e-9)
        fused_emo_text_emb = self.rms_norm(fused_emo_text_emb)

        return fused_emo_text_emb

In [ ]:
class PosteriorNetwork(nn.Module):
    """
    Learns an emotion-aware posterior distribution q(z | x, e)
    over latent space using fused encoder output.
    """
    def __init__(self, input_dim=768, latent_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.GELU(),
            nn.RMSNorm(512),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.RMSNorm(256)
        )
        self.mu_posterior = nn.Linear(256, latent_dim)
        self.logvar_posterior = nn.Linear(256, latent_dim)

    def forward(self, fused_emo_text_emb):
        h = self.mlp(fused_emo_text_emb)
        mu_post = self.mu_posterior(h)
        logvar_post = torch.clamp(self.logvar_posterior(h), min=-5, max=5)

        # Reparameterization trick: sample z ~ N(mu, sigma^2)
        std = torch.exp(0.5 * logvar_post)
        eps = torch.randn_like(std)
        z = mu_post + eps * std

        return z, mu_post, logvar_post

In [ ]:
class PriorNetwork(nn.Module):
    """
    Learns a prior distribution p(z | x)
    based only on text (without emotion labels).
    """
    def __init__(self, input_dim=768, latent_dim=128):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.GELU(),
            nn.RMSNorm(512),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.RMSNorm(256)
        )
        self.mu_prior = nn.Linear(256, latent_dim)
        self.logvar_prior = nn.Linear(256, latent_dim)

    def forward(self, text_emb):
        h = self.mlp(text_emb)
        mu_prior = self.mu_prior(h)
        logvar_prior = torch.clamp(self.logvar_prior(h), min=-5, max=5)

        return mu_prior, logvar_prior

In [16]:
class Classifier(nn.Module):
    """
    Final shared emotion classifier layer
    """
    def __init__(self, latent_dim=128, num_classes=28):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.GELU(),
            nn.LayerNorm(256),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.LayerNorm(128),
            nn.Dropout(0.25),
            nn.Linear(128, num_classes)
        )

    def forward(self, z):
        return self.mlp(z)

In [ ]:
# Main Model class
class EmoAxis(nn.Module):
    def __init__(self, encoder, cross_atten_module, posterior_net, prior_net, classifier):
        """
        Architecture:
        - EncoderBlock   → produces text and fused(text+emotion) embeddings
        - PosteriorNet   → q(z|x,e)
        - PriorNet       → p(z|x)
        - EmotionClassifier → predicts emotions from latent z
        """
        super().__init__()
        self.encoder = encoder
        self.cross_atten = cross_atten_module
        self.posterior = posterior_net
        self.prior = prior_net
        self.classifier = classifier


    def total_params(self):
        """Utility function to check trainable vs total params."""
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Total parameters: {total:,}")
        print(f"Trainable parameters: {trainable:,}")


    def forward(self, input_ids, atten_mask, emotion_bank):
        # Encoder
        encoder_outputs = self.encoder(
            inputs = {"input_ids": input_ids, "attention_mask": atten_mask},
        )
        text_emb = encoder_outputs["text_embed"]                  # [B, H]

        # Cross Attention
        fused_emb = self.cross_atten(encoder_outputs, emotion_bank)    # [B, H]

        # Posterior Net
        z_post, mu_post, logvar_post = self.posterior(fused_emb)

        # Prior Net
        mu_prior, logvar_prior = self.prior(text_emb)
        z_prior = mu_prior + torch.exp(0.5 * logvar_prior) * torch.randn_like(mu_prior)

        # 4. Classifier
        logits_post = self.classifier(z_post)           # from sampled posterior
        logits_prior = self.classifier(z_prior)         # from prior mean

        return {
            "mu_post": mu_post,
            "logvar_post": logvar_post,
            "mu_prior": mu_prior,
            "logvar_prior": logvar_prior,
            "logits_post": logits_post,
            "logits_prior": logits_prior
        }


    def inference(self, input_ids, atten_mask):
        """
        Predict emotions from raw text (inference using prior p(z|x)).
        """
        self.eval()
        with torch.no_grad():
            encoder_outputs = self.encoder(
                inputs = {"input_ids": input_ids, "attention_mask": atten_mask},
            )
            # get only text embeddings
            text_emb = encoder_outputs["text_embed"]

            # latent mean from prior
            mu_prior, _ = self.prior(text_emb)

            # classification
            logits = self.classifier(mu_prior)

            return logits

In [ ]:
encoder = Encoder(base_encoder=roberta_base)

cross_atten_module = CrossAttentionModule()
posterior_net = PosteriorNetwork()
prior_net = PriorNetwork()
classifier = Classifier()

# Initialize model
model = EmoAxis(
    encoder=encoder,
    cross_atten_module=cross_atten_module,
    posterior_net=posterior_net,
    prior_net=prior_net,
    classifier=classifier
)

model.total_params()

Total parameters: 127,674,012
Trainable parameters: 127,674,012


## Loss function

In [ ]:
def kl_divergence(mu_post, logvar_post, mu_prior, logvar_prior):
    """
    KL(N(mu_q, var_q) || N(mu_p, var_p)) averaged over batch.
    Uses diagonal covariance (logvar = log(sigma^2))

    Formula: 0.5 * sum( log(var_p/var_q) + (var_q + (mu_q-mu_p)^2)/var_p - 1 )
    """
    term = logvar_prior - logvar_post + (torch.exp(logvar_post) + (mu_post - mu_prior) ** 2) / torch.exp(logvar_prior) - 1.0
    kl = 0.5 * torch.sum(term, dim=1)

    return kl.mean()


def compute_loss(logits_post, logits_prior, mu_post, logvar_post,
                 mu_prior, logvar_prior, hard_target, soft_target,
                 epoch, lambda_soft=1.0, lambda_kl=0.1):

    # BCE - supervised (posterior only)
    if epoch < 2:
        loss_bce_post = F.binary_cross_entropy_with_logits(logits_post, hard_target)
        loss_bce = loss_bce_post
    else:
        loss_bce_post = F.binary_cross_entropy_with_logits(logits_post, hard_target)
        loss_bce_prior = F.binary_cross_entropy_with_logits(logits_prior, hard_target)
        loss_bce = 0.5 * loss_bce_post + 0.5 * loss_bce_prior

    #Soft MSE - posterior and prior (sigmoid outputs)
    probs_post = torch.sigmoid(logits_post)
    loss_soft_post = F.mse_loss(probs_post, soft_target)
    if epoch < 1:
        loss_soft = loss_soft_post
    else:
        probs_prior = torch.sigmoid(logits_prior)
        loss_soft_prior = F.mse_loss(probs_prior, soft_target)
        loss_soft = loss_soft_post + loss_soft_prior

    # KL Divergence between posterior and prior
    loss_kl = kl_divergence(mu_post, logvar_post, mu_prior, logvar_prior)

    # Total loss
    total_loss = loss_bce + lambda_soft * loss_soft + lambda_kl * loss_kl

    return total_loss, {"loss_bce": loss_bce.item(), "loss_soft": loss_soft.item(), "loss_kl": loss_kl.item()}

## Model Training

In [20]:
def freeze_encoder_layers(encoder, freeze_upto: int=0):
    roberta_base_model = encoder.encoder

    for name, param in roberta_base_model.named_parameters():
        param.requires_grad = True

    if freeze_upto >= 0:
        for layer_idx in range(freeze_upto + 1):
            for param in roberta_base_model.encoder.layer[layer_idx].parameters():
                param.requires_grad = False

    print(f"\n\nFrozen encoder layers - 0 to {freeze_upto}\n\n")


In [21]:
def validate(model, val_dataloader, device, threshold=0.2):
    model.eval()
    preds_all = []
    truths_all = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['atten_masks'].to(device)
            hard_target = batch['hard_target'].to(device)

            # inference
            logits = model.inference(input_ids, attention_mask)

            probs = torch.sigmoid(logits).cpu().numpy()
            preds = (probs >= threshold).astype(int)
            truths = hard_target.cpu().numpy().astype(int)

            preds_all.append(preds)
            truths_all.append(truths)

    preds_all = np.concatenate(preds_all, axis=0)
    truths_all = np.concatenate(truths_all, axis=0)

    micro_f1 = f1_score(truths_all, preds_all, average='micro', zero_division=0)
    macro_f1 = f1_score(truths_all, preds_all, average='macro', zero_division=0)

    return {"micro_f1": micro_f1, "macro_f1": macro_f1}

In [ ]:
def train(
    model: nn.Module,
    train_dataloader: DataLoader,
    val_dataloader: DataLoader,
    device: torch.device,
    epochs: int = 10,
    lr_encoder: float = 5e-6,
    lr_other: float = 3e-5,
    weight_decay: float = 0.01,
    warmup_ratio: float = 0.1,
    kl_anneal_ratio: float = 0.25,
    gradient_accumulation_steps: int = 4,
    max_grad_norm: float = 1.0,
    use_amp: bool = True,
    early_stopping_patience: int = 3,
    min_epochs_before_stop: int = 3
):
    model.to(device)

    steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    total_steps = steps_per_epoch * epochs

    warmup_steps = int(total_steps * warmup_ratio)

    global_step = 0
    best_val_microF1 = -1.0
    epochs_no_improve = 0
    current_freeze_config = None

    for epoch in range(0, epochs):
        model.train()
        epoch_loss = 0.0

        if epoch < 2:
            freeze_level = 5
            freeze_desc = 'Training last 6 layers'
        elif epoch < 4:
            freeze_level = 3
            freeze_desc = 'Training last 8 layers'
        else:
            freeze_level = -1
            freeze_desc = 'Training all layers'

        if freeze_level != current_freeze_config:
            freeze_encoder_layers(encoder=model.encoder, freeze_upto=freeze_level)
            current_freeze_config = freeze_level

            encoder_params, other_params = [], []
            for name, p in model.named_parameters():
                if not p.requires_grad:
                    continue
                if 'encoder' in name:
                    encoder_params.append(p)
                else:
                    other_params.append(p)

            optimizer = AdamW([
                {"params": encoder_params, "lr": lr_encoder},
                {"params": other_params, "lr": lr_other}
            ], weight_decay=weight_decay)

            scheduler = get_cosine_schedule_with_warmup(
                        optimizer,
                        num_warmup_steps=warmup_steps,
                        num_training_steps=total_steps
                    )

        scaler = GradScaler(enabled=(use_amp and device.type =='cuda'))

        print(f"--- Epoch {epoch+1}/{epochs} ({freeze_desc}) ---")

        optimizer.zero_grad()

        for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['atten_masks'].to(device)
            hard_target = batch['hard_target'].to(device)
            soft_target = batch['soft_target'].to(device)

            with autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
                outputs = model(input_ids=input_ids,
                                atten_mask=attention_mask,
                                emotion_bank=emotion_bank)

                logits_post = outputs['logits_post']
                logits_prior = outputs['logits_prior']
                mu_post, logvar_post = outputs['mu_post'], outputs['logvar_post']
                mu_prior, logvar_prior = outputs['mu_prior'], outputs['logvar_prior']

                kl_weight = float(torch.sigmoid(torch.tensor((global_step - kl_anneal_steps/2) / (kl_anneal_steps/10))))

                total_loss, _ = compute_loss(
                    logits_post, logits_prior,
                    mu_post, logvar_post, mu_prior, logvar_prior,
                    hard_target, soft_target,
                    epoch=epoch, lambda_kl=kl_weight
                )
                total_loss = total_loss / gradient_accumulation_steps

            scaler.scale(total_loss).backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

            epoch_loss += total_loss.item() * gradient_accumulation_steps
        avg_loss = epoch_loss / len(train_dataloader)
        print(f"Epoch {epoch+1}/{epochs} | Avg Loss: {avg_loss:.4f}")

        metrics = validate(model, val_dataloader, device)
        micro_F1 = metrics.get("micro_f1", -1)
        macro_F1 = metrics.get("macro_f1", -1)
        print(f"Validation: micro-F1 = {micro_F1:.4f}, macro-F1 = {macro_F1:.4f}")

        if micro_F1 > best_val_microF1:
            best_val_microF1 = micro_F1
            epochs_no_improve = 0
            os.makedirs("model_dir", exist_ok=True)
            torch.save(model.state_dict(), os.path.join("/content/model_dir", "best_model.pt"))
            print("Micro-F1 score improved — model saved.")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epoch + 1 >= min_epochs_before_stop and epochs_no_improve >= early_stopping_patience:
            print("\nEarly stopping activated.")
            break

    print(f"\n\nTraining completed. Best validation micro-F1 = {best_val_microF1:.4f}")
    return model


In [23]:
!nvidia-smi

Wed Oct 29 10:45:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
import torch
print(torch.cuda.is_available())  # Should be True
device_name=torch.cuda.get_device_name(0)
print(device_name) # Should say 'Tesla T4'


True
Tesla T4


In [25]:
# SET DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train(model=model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, device=device)

Using device: cuda


Frozen encoder layers - 0 to 5


--- Epoch 1/10 (Training last 6 layers) ---
Epoch 1/10 | Avg Loss: 0.7386
Validation: micro-F1 = 0.2257, macro-F1 = 0.0403
Micro-F1 score improved — model saved.
--- Epoch 2/10 (Training last 6 layers) ---
Epoch 2/10 | Avg Loss: 0.4401
Validation: micro-F1 = 0.1121, macro-F1 = 0.0135
No improvement for 1 epoch(s).


Frozen encoder layers - 0 to 3


--- Epoch 3/10 (Training last 8 layers) ---
Epoch 3/10 | Avg Loss: 0.4251
Validation: micro-F1 = 0.2263, macro-F1 = 0.0404
Micro-F1 score improved — model saved.
--- Epoch 4/10 (Training last 8 layers) ---
Epoch 4/10 | Avg Loss: 0.4251
Validation: micro-F1 = 0.2263, macro-F1 = 0.0404
No improvement for 1 epoch(s).


Frozen encoder layers - 0 to -1


--- Epoch 5/10 (Training all layers) ---
Epoch 5/10 | Avg Loss: 0.4197
Validation: micro-F1 = 0.2263, macro-F1 = 0.0404
No improvement for 2 epoch(s).
--- Epoch 6/10 (Training all layers) ---
Epoch 6/10 | Avg Loss: 0.4208
Validation: micro-F1 

EmoAxis(
  (encoder): Encoder(
    (encoder): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_features=768, o